In [1]:
import igibson.object_states as object_states
from igibson.tasks.behavior_task import BehaviorTask
from igibson.utils.ig_logging import IGLogReader
from igibson.utils.utils import parse_config
import os
import igibson
from igibson.envs.igibson_env import iGibsonEnv


 _   _____  _  _
(_) / ____|(_)| |
 _ | |  __  _ | |__   ___   ___   _ __
| || | |_ || || '_ \ / __| / _ \ | '_ \
| || |__| || || |_) |\__ \| (_) || | | |
|_| \_____||_||_.__/ |___/ \___/ |_| |_|



torch is not available, falling back to rendering to memory (instead of tensor)


In [2]:
class Env:

    def defalt_init(self,demo_path):
        task = IGLogReader.read_metadata_attr(demo_path, "/metadata/atus_activity")
        if task is None:
            task = IGLogReader.read_metadata_attr(demo_path, "/metadata/task_name")

        task_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/activity_definition")
        if task_id is None:
            task_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/task_instance")

        scene_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/scene_id")

        config_filename = os.path.join(igibson.configs_path, "behavior_robot_mp_behavior_task.yaml")
        config = parse_config(config_filename)
        

        config["task"] = task
        config["task_id"] = task_id
        config["scene_id"] = scene_id
        config["robot"]["show_visual_head"] = True
        config["image_width"]=512
        config["image_height"]=512
        self.config = config
    
            
    def __init__(self,demo_path=None) -> None:
        self.config=None
        if demo_path is not None:
            self.defalt_init(demo_path)

    

In [3]:
demo_path=rb"D:\GitHub\behavior-vllm-eval\igibson\data\virtual_reality\bottling_fruit_0_Wainscott_0_int_0_2021-05-24_19-46-46.hdf5"
eenv=Env(demo_path=demo_path)
igenv = iGibsonEnv(config_file=eenv.config)
bt=BehaviorTask(igenv)

KeyboardInterrupt: 

In [ ]:
for name, obj in bt.object_scope.items():
    print(name,obj.name)

strawberry.n.01_1 strawberry_0_multiplexer
electric_refrigerator.n.01_1 fridge_97
peach.n.03_1 peach_0_multiplexer
countertop.n.01_1 countertop_84
jar.n.01_1 jar_0
jar.n.01_2 jar_1
carving_knife.n.01_1 carving_knife_0
cabinet.n.01_1 bottom_cabinet_no_top_80
floor.n.01_1 room_floor_kitchen_0
agent.n.01_1 agent_2BD36CE6280


In [ ]:
from igibson.objects.multi_object_wrappers import ObjectMultiplexer,ObjectGrouper
from igibson.objects.articulated_object import URDFObject
from igibson.object_states.on_floor import RoomFloor
name_mapping={}
for name, obj in bt.object_scope.items():
    if isinstance(obj, ObjectMultiplexer):
        name_mapping[name]=obj.name.rstrip("_multiplexer")
    elif isinstance(obj, RoomFloor) or isinstance(obj, URDFObject):
        name_mapping[name]=obj.name

In [ ]:
print(name_mapping)

{'strawberry.n.01_1': 'strawberry_0', 'electric_refrigerator.n.01_1': 'fridge_97', 'peach.n.03_1': 'peach_0', 'countertop.n.01_1': 'countertop_84', 'jar.n.01_1': 'jar_0', 'jar.n.01_2': 'jar_1', 'carving_knife.n.01_1': 'carving_knife_0', 'cabinet.n.01_1': 'bottom_cabinet_no_top_80', 'floor.n.01_1': 'room_floor_kitchen_0'}


In [ ]:
for name, obj in bt.object_scope.items():
    print(name,obj.name)

strawberry.n.01_1 strawberry_0_multiplexer
electric_refrigerator.n.01_1 fridge_97
peach.n.03_1 peach_0_multiplexer
countertop.n.01_1 countertop_84
jar.n.01_1 jar_0
jar.n.01_2 jar_1
carving_knife.n.01_1 carving_knife_0
cabinet.n.01_1 bottom_cabinet_no_top_80
floor.n.01_1 room_floor_kitchen_0
agent.n.01_1 agent_2BD36CE6280


In [ ]:
for initial_cond in bt.initial_conditions:
    print(initial_cond.terms)

['inside', 'strawberry.n.01_1', 'electric_refrigerator.n.01_1']
['inside', 'peach.n.03_1', 'electric_refrigerator.n.01_1']
['not', 'sliced', 'strawberry.n.01_1']
['not', 'sliced', 'peach.n.03_1']
['ontop', 'jar.n.01_1', 'countertop.n.01_1']
['ontop', 'jar.n.01_2', 'countertop.n.01_1']
['ontop', 'carving_knife.n.01_1', 'countertop.n.01_1']
['onfloor', 'agent.n.01_1', 'floor.n.01_1']


In [ ]:
objects=""
for name in name_mapping.values():
    objects+=name+"\n"
print(objects)

strawberry_0
fridge_97
peach_0
countertop_84
jar_0
jar_1
carving_knife_0
bottom_cabinet_no_top_80
room_floor_kitchen_0



In [ ]:
for goal_cond in bt.initial_conditions:
    a=goal_cond.terms
    b=[]
    for name in a:
        if name in name_mapping:
            b.append(name_mapping[name])
        else:
            b.append(name)
    print(b)

['inside', 'strawberry_0', 'fridge_97']
['inside', 'peach_0', 'fridge_97']
['not', 'sliced', 'strawberry_0']
['not', 'sliced', 'peach_0']
['ontop', 'jar_0', 'countertop_84']
['ontop', 'jar_1', 'countertop_84']
['ontop', 'carving_knife_0', 'countertop_84']
['onfloor', 'agent.n.01_1', 'room_floor_kitchen_0']


In [ ]:
for goal_cond in bt.goal_conditions:
    a=goal_cond.terms
    b=[]
    for name in a:
        if name in name_mapping:
            b.append(name_mapping[name])
        else:
            b.append(name)
    print(b)

['exists', 'jar.n.01', '-', 'jar.n.01', 'and', 'inside', 'strawberry_0', 'jar.n.01', 'not', 'inside', 'peach_0', 'jar.n.01']
['exists', 'jar.n.01', '-', 'jar.n.01', 'and', 'inside', 'peach_0', 'jar.n.01', 'not', 'inside', 'strawberry_0', 'jar.n.01']
['forall', 'jar.n.01', '-', 'jar.n.01', 'not', 'open', 'jar.n.01']
['sliced', 'strawberry_0']
['sliced', 'peach_0']


In [ ]:
bt.initial_conditions[7].terms

['onfloor', 'agent.n.01_1', 'floor.n.01_1']

In [ ]:
bt.goal_conditions

In [ ]:
print(bt.goal_conditions[0].terms)

['exists', 'jar.n.01', '-', 'jar.n.01', 'and', 'inside', 'strawberry.n.01_1', 'jar.n.01', 'not', 'inside', 'peach.n.03_1', 'jar.n.01']
